This section shows different ways to validate your function arguments

In [1]:
from vpfargs import validate

You can ensure that the function arguments have the correct type

In [2]:
@validate(int, float, bool)
def foo(x, y, z):
    print(type(x), type(y), type(z))

foo(1, 1.0, True)
try:
    foo(1, 1.0, None)
except Exception as e:
    print(e)

<class 'int'> <class 'float'> <class 'bool'>
Invalid argument at position 3: Type bool expected but got NoneType


Some argument could have different valid types...

In [3]:
from vpfargs import number

@validate([str, float, bool], number)
def bar(x, y):
    pass

bar('Hello World!', 1.0)
bar(1.0, 1.0)
bar(True, 1)
try:
    bar(None, 1.0)
except Exception as e:
    print(e)

Invalid argument at position 1: Type bool or float or str expected but got NoneType


In the above code, the first argument must be an str, float or bool object, whereas the second argument could be any numeric type (int, float, Decimal)

You can pass your classes as validators. The function will expect an instance of the class you specified or subclass of it.

In [4]:
class Animal:
    pass

class Cat(Animal):
    def __str__(self):
        return 'cat'

class Dog(Animal):
    def __str__(self):
        return 'dog'

@validate(Animal, Cat, Dog)
def foo(x, y, z):
    print(', '.join([str(x), str(y), str(z)]))
          
foo(Cat(), Cat(), Dog())
foo(Dog(), Cat(), Dog())
try:
    foo(Cat(), Dog(), Cat())
except Exception as e:
    print(e)

cat, cat, dog
dog, cat, dog
Invalid argument at position 2: Type Cat expected but got Dog


Class enumerations can also work as regular validators...

In [5]:
from enum import Enum, auto

class Color(Enum):
    RED = auto(),
    GREEN = auto(),
    BLUE = auto()

@validate(Color)
def bar(x):
    print(x)
    
bar(Color.RED)
bar(Color.GREEN)
bar(Color.BLUE)
try:
    bar(False)
except Exception as e:
    print(e)

Color.RED
Color.GREEN
Color.BLUE
Invalid argument at position 1: Type Color expected but got bool


You can ensure that your arguments have a correct type and also they take a discrete value within a numeric range, list, ...

In [6]:


@validate(['r', 'w', 'x'], range(0, 20), (0, 1, 'yes', 'no'))
def foo(x, y, z):
    print(x, y, z)

foo('r', 10, 0)
foo('w', 15, 'yes')
try:
    foo('r', None, 0)
except Exception as e:
    print(e)

try:
    foo('r', 10, 'maybe')
except Exception as e:
    print(e)

r 10 0
w 15 yes
Invalid argument at position 2: Value in [0, 1, ..., 18, 19] expected but got None
Invalid argument at position 3: Value in [0, 1, 'yes', 'no'] expected but got maybe


In the previous code, the first argument must be either 'r', 'w' or 'x' (all string values)
The second argument must be an int object within the range [0, 20) and the last argument could be 0, 1, 'yes' or 'no' (either ints or strings)
  

You can combine your validators grouping them in a tuple or a list

In [7]:
@validate([str, range(0, 10)])
def foo(x):
    print(x)

foo(5)
foo('Hello World!')
try:
    foo(True)
except Exception as e:
    print(e)

5
Hello World!
Invalid argument at position 1: Expected value of type str or int value in [0, 1, ..., 8, 9] but got True (bool type)


x will match any str OR a int object in the range [0, 10)

You can define your custom validators as regular methods or lambdas....

In [8]:
def even(x):
    return x % 2 == 0

odd = lambda x: x % 2 == 1

@validate(even, odd)
def foo(x, y):
    print(x % 2, y % 2)
    
foo(0, 1)
foo(2, 5)
try:
    foo(1, 1)
except Exception as e:
    print(e)
    
try:
    foo(0, 2)
except Exception as e:
    print(e)

0 1
0 1
Invalid argument at position 1: Expression even(1) evaluated to False
Invalid argument at position 2: Expression evaluated to False


Your validator must return any object that python evaluates to true if the argument is valid or something that evaluates to false otherwise.

You can raise an exception inside your validator instead of returning False to append some text to the error message. e.g:

In [15]:
def even(x):
    if x % 2 > 0:
        raise Exception('{} is not even'.format(x))
    return True

@validate(even)
def foo(x):
    print(x % 2)

foo(0)
try:
    foo(5)
except Exception as e:
    print(e)

0
Invalid argument at position 1: 5 is not even


Validators can be stacked (You can define multiple validation stages)


In [18]:
foo(2.0)

0.0


Suppose that we want to consider only int values whose modulus (operation %) with 2 is 0 (even natural numbers).
We can make a first a type check (to ensure that only int values are valid)

In [23]:
@validate(int)
@validate(even)
def foo(x):
    print(type(x).__name__, x % 2)
    
foo(2)
try:
    foo(2.0)
except Exception as e:
    print(e)
    
try:
    foo(3)
except Exception as e:
    print(e)

int 0
Invalid argument at position 1: Type int expected but got float (at level 1)
Invalid argument at position 1: 3 is not even (at level 2)
